In [1]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
from pathlib import Path
import pandas as pd

from typing import Optional, Any
import requests
from mlscraper.html import Page
from mlscraper.samples import Sample, TrainingSet
from mlscraper.training import train_scraper
import pickle
import extruct
import datetime

In [19]:
from pybloom import ScalableBloomFilter
import os
import pickle



# bloom_path = "/bloom"

class BloomFilter():
    def __init__(self):
        if os.path.isfile("bloom_filter"):
            with open("bloom_filter", "rb") as f:
                self.bloom = pickle.load(f)
        else:
            self.bloom = ScalableBloomFilter(mode=ScalableBloomFilter.SMALL_SET_GROWTH, error_rate=0.001)

    def save(self):
        # if not os.path.exists(bloom_path):
        #     os.makedirs(bloom_path)
        with open("bloom_filter", "wb") as f:
            pickle.dump(self.bloom, f, protocol=pickle.HIGHEST_PROTOCOL)

    def check_bloom_filter(self, text: str) -> bool:
        text = text.strip().lower()
        res = text in self.bloom
        if not res:
            self.bloom.add(text)
        return res

In [31]:
es_ache = Elasticsearch("http://elasticsearch-clean:9200")
index="clean_index_apr"
print(es_ache.cluster.health())

{'cluster_name': 'elasticsearch-clean', 'status': 'green', 'timed_out': False, 'number_of_nodes': 3, 'number_of_data_nodes': 3, 'active_primary_shards': 3, 'active_shards': 6, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [70]:
# animals =[]
# with open("seeds_animals_cites.txt", "r") as f:
#     for line in f:
#         animals.append(line.strip())

In [72]:
def open_scrap(domain):
    site = "scraper_"+domain
    with open(site, "rb") as f:
        scraper = pickle.load(f)
    return scraper

In [73]:
scraper = open_scrap("ebay")

In [74]:
http = "https://www.ebay.com/itm/384693477548?hash=item599184a0ac:g:0jsAAOSwTHNiTWNa&amdata=enc%3AAQAIAAAA4GBg9A4c7Z11DQDr0I1DAWh864ozZtXprcOzEjdCkvxi12FY1VCl9UgflOOMVMqWUG0B9InP12Y2H6Ksd4dR4uzPC0C6REnu26XTuPc6hUHzRkQSX0lQwt6E1RQxFiOyOEk94oNDaexr1xgzF1SenL%2FBlgviAW1poF1rXEBsHaoWTvfyYAqZ3OgmNjUEeIKbF7PrnAsySKgkKAMEbdPJ6MYYXxQO7GTJlrmn5OH55%2BLU2nsutqWUPYDBhioMSh3kk%2FM1HJRZ5z5vcMjHOWEwxTfzqO%2B2CL2WixXuJcchmyzg%7Ctkp%3ABk9SR6K694j9YQ"
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
resp = requests.get(http, headers= headers)
test = scraper.get(Page(resp.content))
test

{'product': 'Animal Insect Enamel Pin Lapel Collar Pin Corsage Brooch Women Men Jewelry Gift  | eBay',
 'seller': 'yaojie_80',
 'ships to': 'Worldwide',
 'price': '0.99',
 'location': '义乌, China'}

In [ ]:
## observation
## add seller page url

In [10]:
domain_scrapers = ['gumtree',
 'auctionzip',
 '1stdibs',
 'canadianlisted',
 'stsy',
 'ukclassifieds',
 'taxidermy',
 'skullsunlimited',
 'glacierwear',
 'picclick',
 'preloved',
 'ebay']

In [11]:
def get_sintax_opengraph(metadata: dict):
    product_meta = {}
    if product_meta["price"]:
        product_meta["name"] = metadata.get("og:title")
        product_meta["description"]=  metadata.get("og:description")
        product_meta["price"] = metadata.get("product:price:amount")
        product_meta["currency"] =  metadata.get("product:price:currency")
        product_meta["category"] = metadata.get("product:category")
    
    return product_meta


def get_sintax_dublincore(metadata: dict):
    product_meta = {}
    metas = metadata.get("elements")
    for meta in metas:
        if meta.get("name") == "description":
            if meta.get("price"):
                product_meta["name"] = meta.get("name")
                product_meta["description"]=  meta.get("content")
                product_meta["price"] = meta.get("price")
                product_meta["currency"] =  meta.get("currency")
                product_meta["category"] = meta.get("category")
    return product_meta

def get_dict_json_ld(meta):
    # dict_page = []
    product = meta
    # for product in meta:
    obj = {}
    obj["name"]= product.get("name")
    obj["description"]= product.get("description")
    obj["image"]= product.get("image")
    if obj["image"] and isinstance(obj["image"], dict):
        obj["image"]= product.get("image").get("contentUrl")
    if obj["image"] and isinstance(obj["image"], list):
        obj["image"]= product.get("image")[0]
        if obj["image"] and isinstance(obj["image"], dict):
            obj["image"]= obj["image"].get("contentUrl")
    obj["url"]= product.get("url")
    obj["production_data"]= product.get("productionDate")
    obj["category"]= product.get("category")
    if product.get("offers"):
        obj["price"]= product.get("offers").get("price")
        obj["currency"]= product.get("offers").get("priceCurrency")
        if product.get("offers").get("seller"):
            obj["seller"]= product.get("offers").get("seller").get("name")
            obj["seller_type"]= product.get("offers").get("seller").get("@type")
            if product.get("offers").get("seller").get("address"):
                # obj["address_type"]= product.get("offers").get("seller").get("address").get("@type")
                # obj["addres_location"]= product.get("offers").get("seller").get("address").get("addressLocality")
                obj["location"]= product.get("offers").get("seller").get("address").get("addressCountry")
    # dict_page.append(obj)
    return obj

def get_dict_microdata(meta):
    dict_page = []
    for product in meta:
        obj = {}
        obj["name"]= product.get("name")
        obj["description"]= product.get("description")
        obj["image"]= product.get("image")
        if obj["image"] and isinstance(obj["image"], dict):
            obj["image"]= obj["image"].get("contentUrl")
        if obj["image"] and isinstance(obj["image"], list):
            obj["image"]= product.get("image")[0]
            if obj["image"] and isinstance(obj["image"], dict):
                 obj["image"]= obj["image"].get("contentUrl")
        obj["production_data"]= product.get("productionDate")
        obj["category"]= product.get("category")
        if product.get("offers") and isinstance(product.get("offers"), dict):
            obj["url"]= product.get("offers").get("url")
            obj["price"]= product.get("offers").get("price")
            obj["currency"]= product.get("offers").get("priceCurrency")
            if product.get("offers").get("seller"):
                obj["seller"]= product.get("offers").get("seller").get("name")
                obj["seller_type"]= product.get("offers").get("seller").get("@type")
                if product.get("offers").get("seller").get("address"):
                    # obj["address_type"]= product.get("offers").get("seller").get("address").get("@type")
                    # obj["addres_location"]= product.get("offers").get("seller").get("address").get("addressLocality")
                    obj["location"]= product.get("offers").get("seller").get("address").get("addressCountry")
        elif product.get("offers") and isinstance(product.get("offers"), list):
            obj["url"]= product.get("offers")[0].get("url")
            obj["price"]= product.get("offers")[0].get("price")
            obj["currency"]= product.get("offers")[0].get("priceCurrency")
            if product.get("offers")[0].get("seller"):
                obj["seller"]= product.get("offers")[0].get("seller").get("name")
                obj["seller_type"]= product.get("offers")[0].get("seller").get("@type")
                if product.get("offers")[0].get("seller").get("address"):
                    # obj["address_type"]= product.get("offers")[0].get("seller").get("address").get("@type")
                    # obj["addres_location"]= product.get("offers")[0].get("seller").get("address").get("addressLocality")
                    obj["location"]= product.get("offers")[0].get("seller").get("address").get("addressCountry")
        dict_page.append(obj)
    return dict_page


In [41]:
def create_df(ads):
    df_metas = pd.DataFrame()
    for ad in ads:
        scraper = None
        data_ext = []
        metas = {}
        domain = ad["_source"]["domain"].split(".")[0]
        try:
            if domain in domain_scrapers:
                scraper = open_scrap(domain)
                data_ext = scraper.get(Page(ad["_source"]["html"]))
                if data_ext["product"]:
                    data_ext["name"] = data_ext.pop("product")
                meta_df = pd.DataFrame(data_ext, index=[0])
                meta_df["url"] = ad["_source"]["url"]
                meta_df["title"] = ad["_source"]["title"]
                meta_df["text"] = ad["_source"]["text"]
                df_metas = pd.concat([df_metas, meta_df])
        except Exception:
            data_ext = []
        try:
            metadata = extruct.extract(ad["_source"]["html"], 
                                               base_url=ad["_source"]["url"],
                                               uniform=True,
                                               syntaxes=['json-ld',
                                                         'microdata',
                                                         'opengraph',
                                                        'dublincore'])

            if metadata.get("microdata"):
                meta_df = pd.DataFrame(get_dict_microdata(metadata.get("microdata")))
                meta_df["url"] = ad["_source"]["url"]
                meta_df["title"] = ad["_source"]["title"]
                meta_df["text"] = ad["_source"]["text"]
                df_metas = pd.concat([df_metas, meta_df])
            if metadata.get("json-ld"):
                for meta in metadata.get("json-ld"):
                    if meta.get("@type") == 'Product':
                        meta_dict = get_dict_json_ld(meta)
                        meta_df = pd.DataFrame([meta_dict])
                        meta_df["url"] = ad["_source"]["url"]
                        meta_df["title"] = ad["_source"]["title"]
                        meta_df["text"] = ad["_source"]["text"]
                        df_metas = pd.concat([df_metas, meta_df])
            if metadata.get("opengraph"):
                open_ = get_sintax_dublincore(metadata.get("dublincore")[0])
                if open_:
                    meta_df = pd.DataFrame(open_, index=[0])
                    meta_df["url"] = ad["_source"]["url"]
                    meta_df["title"] = ad["_source"]["title"]
                    meta_df["text"] = ad["_source"]["text"]
                    df_metas = pd.concat([df_metas, meta_df])


                # metas["metas_opengraph"] = get_sintax_opengraph(metadata.get("opengraph")[0])
                # meta_df = pd.DataFrame(get_sintax_opengraph(metadata.get("opengraph")[0]), index=[0])
                # meta_df["url"] = ad["_source"]["url"]
                # meta_df["title"] = ad["_source"]["title"]
                # meta_df["text"] = ad["_source"]["text"]
            if metadata.get("dublincore"):
                dublin = get_sintax_dublincore(metadata.get("dublincore")[0])
                if dublin:
                    meta_df = pd.DataFrame(dublin)
                    meta_df["url"] = ad["_source"]["url"]
                    meta_df["title"] = ad["_source"]["title"]
                    meta_df["text"] = ad["_source"]["text"]
                    df_metas = pd.concat([df_metas, meta_df])
                # metas["metas_dublincore"] = get_sintax_dublincore(metadata.get("dublincore")[0])
        except Exception as e:
            print(e) 
            # raise e

            
    df_metas = df_metas.groupby('url').agg({"name": 'first',
                            "description":'first',
                            "image":'first',
                            "production_data": 'first',
                            "category": 'first',
                            "title":'first',
                            "text":'first',
                            "price":'first',
                            "currency":"first",
                            "seller":'first',
                            "seller_type":'first',
                            "location":'first',
                            "ships to":'first'}).reset_index()
    df_metas = df_metas.astype({'name': 'str', 'price': 'str', 'currency': 'str', 'image': 'str', 'seller': 'str', 'seller_type': 'str', 'location': 'str', 'ships to': 'str' })
 
    ct = datetime.datetime.now()
    file = "data_april14-15/data_"+ct.strftime("%m-%d-%YT%H:%M:%S")
    df_metas.to_parquet(file, index=False)


    #     data.append(pd.DataFrame({"url": ad["_source"]["url"], "title": ad["_source"]["title"], "text": ad["_source"]["text"]}, index=[0]))
    #     # metas_df.append(pd.DataFrame(
    #     #                           "metas": [metas], "mlscraper": [data_ext]}))


In [54]:
docs = scan(
            client=es_ache,
            index=index,
            scroll='25m',
            query= { "query":{
                        "bool": {
                            "filter": {
                                "range": {
                                    "retrieved": {
                                        "gte": "2023-04-13T00:00:00",
                                        "lt": "2023-04-13T06:00:00",
                                    },
                                },
                            },
                        }
                    }
                    })

In [57]:
bloom = BloomFilter()

In [ ]:
## bottleneck

In [61]:
ads = []
count=0
for doc in docs:
    if not bloom.check_bloom_filter(doc["_source"]["url"]):
        ads.append(doc)
        count += 1
        if count ==5000:
            create_df(ads)
            ads=[]
            count=0
            bloom.save()
if ads:
    create_df(ads)
    ads=[]
    count=0
    bloom.save()

In [62]:
print("finished")

finished


In [64]:
docs = scan(
            client=es_ache,
            index=index,
            scroll='25m',
            query= { "query":{
                        "bool": {
                            "filter": {
                                "range": {
                                    "retrieved": {
                                        "gte": "2023-04-13T06:00:00",
                                        "lt": "2023-04-13T12:00:00",
                                    },
                                },
                            },
                        }
                    }
                    })

In [65]:
ads = []
count=0
for doc in docs:
    if not bloom.check_bloom_filter(doc["_source"]["url"]):
        ads.append(doc)
        count += 1
        if count ==5000:
            create_df(ads)
            ads=[]
            count=0
            bloom.save()
if ads:
    create_df(ads)
    ads=[]
    count=0
    bloom.save()
            

Expecting ',' delimiter: line 6 column 24 (char 95)
Expecting ',' delimiter: line 6 column 19 (char 90)
Expecting ',' delimiter: line 6 column 48 (char 119)
Expecting ',' delimiter: line 6 column 39 (char 110)
Expecting ',' delimiter: line 6 column 38 (char 109)
Expecting ',' delimiter: line 13 column 44 (char 736)
Expecting ',' delimiter: line 6 column 31 (char 102)


In [66]:
print("finished")

finished


In [ ]:
## count to see if finish

In [61]:
docs = scan(
            client=es_ache,
            index=index,
            scroll='25m',
            size=10,
            raise_on_error=False,
            query= { "query":{
                        "bool": {
                            "filter": {
                                "range": {
                                    "retrieved": {
                                        "gte": "2023-04-15T06:00:00",
                                        "lt": "2023-04-15T12:00:00",
                                    },
                                },
                            },
                        }
                    }
                    })

In [62]:
bloom = BloomFilter()

In [63]:
ads = []
count=0
for doc in docs:
    if not bloom.check_bloom_filter(doc["_source"]["url"]):
        ads.append(doc)
        count += 1
        if count == 5000:
            create_df(ads)
            ads=[]
            count=0
            bloom.save()
if ads:
    create_df(ads)
    ads=[]
    count=0
    bloom.save()

In [64]:
print("finished")

finished


In [36]:
count

0